In [1]:
import sqlite3
import pandas as pd
import os
import pickle


# Why Python Pickle is used?
The pickle module in Python is used for serializing and deserializing Python objects. Serialization is the process of converting a Python object into a byte stream, which can then be stored in a file, sent over a network, or saved in a database. Deserialization is the reverse process, where the byte stream is converted back into a Python object.

In [2]:
# open the csv file and add a new column with the image link

csv_file = 'data/prova/a.csv'
df = pd.read_csv(csv_file)
df['image_path'] = 'temp_images\AC_Ace_Ace Bristol'
df.to_csv('data/prova/a.csv', index=False)

In [3]:
# In this block we make a new table in the database from the csv file

import subprocess

# Path to the CSV file and database
csv_file = 'data/prova/a.csv'
db_file = 'data/prova/my_cars.db'
table_name = 'my_table'

# Command to import CSV into SQLite
command = f'sqlite3 {db_file} ".mode csv" ".import {csv_file} {table_name}"'

# Execute the command
subprocess.run(command, shell=True, check=True)

CompletedProcess(args='sqlite3 data/prova/my_cars.db ".mode csv" ".import data/prova/a.csv my_table"', returncode=0)

In [4]:
# Function to read image from path and convert to binary and serialize it to store in the database
def images_to_binary(images_dir):
    binary_images = []
    for image_path in os.listdir(images_dir):
        image_path = os.path.join(images_dir, image_path)
        with open(image_path.strip(), 'rb') as file:
            binary_images.append(file.read())
    return pickle.dumps(binary_images)  # Serialize the list of binary images


In [ ]:
df = pd.read_csv('data/prova/a.csv')
df['images'] = df['image_path'].apply(images_to_binary)

# TO SOLVE BEFORE ####################
# rename the column to avoid conflict with the column names in the database
df = df.rename(columns={"power": 'power_1'})
df = df.rename(columns={"length": 'length_1'})
df = df.rename(columns={"width": 'width_1'})
######################################

# Connessione al database SQLite (crea il database se non esiste)
conn = sqlite3.connect(db_file)

# Save the dataframe to a SQLite table
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Commit the changes
conn.commit()
conn.close()

In [9]:
def save_first_image(serialized_images, output_dir='output_images', output_filename='retrieved_image.jpg'):
    # Deserialize the images
    images = pickle.loads(serialized_images)
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Define the output path
    output_path = os.path.join(output_dir, output_filename)
    
    # Save the first image to a file
    with open(output_path, 'wb') as file:
        file.write(images[0])
    
    print(f'Image saved to {output_path}')

In [ ]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file)

# Read the data from the table into a pandas DataFrame
df_solo = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)


# Close the connection
conn.close()

imgs = df['images'][1]

save_first_image(imgs)